<a href="https://colab.research.google.com/github/r-kovalch/acter-ner/blob/main/notebooks/acter-gliner-small-en.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/r-kovalch/acter-ner

Cloning into 'acter-ner'...
remote: Enumerating objects: 263, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 263 (delta 11), reused 7 (delta 5), pack-reused 231 (from 1)
Receiving objects: 100% (263/263), 2.43 MiB | 3.50 MiB/s, done.
Resolving deltas: 100% (165/165), done.


In [2]:
!git clone https://github.com/AylaRT/ACTER

Cloning into 'ACTER'...
remote: Enumerating objects: 5448, done.
remote: Counting objects: 100% (5448/5448), done.
remote: Compressing objects: 100% (3304/3304), done.
remote: Total 5448 (delta 2684), reused 4893 (delta 2132), pack-reused 0 (from 0)
Receiving objects: 100% (5448/5448), 11.90 MiB | 10.19 MiB/s, done.
Resolving deltas: 100% (2684/2684), done.


In [3]:
%cd /content/ACTER

/content/ACTER


In [4]:
%cd /content/acter-ner/term_extractor

/content/acter-ner/term_extractor


In [5]:
ls

combine_corpora-en-fr-nl.sh  dataset_processor.py        train_full.tsv
combine_corpora-en-fr.sh     output/                     train_model.py
combine_corpora_gliner.sh    preprocess_acter_gliner.py  train_spacy_model.py
combine_corpora.sh           preprocess_acter.py         Untitled.ipynb
configs/                     test_full.tsv


In [6]:
!bash combine_corpora_gliner.sh

In [7]:
!spacy convert --converter iob train_full.tsv output && \
  spacy convert --converter iob test_full.tsv output


ℹ Auto-detected token-per-line NER format
ℹ Grouping every 1 sentences into a document.
⚠ To generate better training data, you may want to group sentences
into documents with `-n 10`.
✔ Generated output file (11329 documents): output/train_full.spacy
ℹ Auto-detected token-per-line NER format
ℹ Grouping every 1 sentences into a document.
⚠ To generate better training data, you may want to group sentences
into documents with `-n 10`.
✔ Generated output file (2833 documents): output/test_full.spacy


In [8]:
!mkdir -p input_data && \
  mv output/train_full.spacy input_data/train_full.spacy && \
  mv output/test_full.spacy input_data/test_full.spacy

In [9]:
!cd input_data && ls

test_full.spacy  train_full.spacy


In [10]:
!pip install 'spacy[transformers]' -q
!pip install -U "gliner-spacy>=0.0.11"  -q # latest release, Jan 2025
!pip install -U "gliner>=0.2.5"        -q  # core GLiNER lib


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.2/756.2 kB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 124.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 99.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [11]:
# !spacy debug data configs/config_gliner_small.cfg --verbose

In [12]:
import spacy

from spacy.tokens import DocBin
db = DocBin().from_disk("./input_data/train_full.spacy")
docs = list(db.get_docs(spacy.blank("en").vocab))
print(docs[0].text, docs[0].ents)


.  ()


In [ ]:
!spacy train configs/config_gliner_small.cfg --output ./model --gpu-id 0

✔ Created output directory: model
ℹ Saving to output directory: model
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
2025-05-12 13:52:50.847803: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747057970.868771    2209 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747057970.875149    2209 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Fetching 10 files:   0% 0/10 [00:00<?, ?it/s]
spm.model: 100% 2.46M/2.46M [00:00<00:00, 90.5MB/s]

gliner_config.json: 100% 676/676 [00:00<00:00, 4.63MB/s]

added_tokens.json: 100% 65.0/65.0 [00:00<00:00, 645kB/s]

special_tokens_map.json: 100% 970/970 [00:00<00:00, 9.87MB

In [ ]:
!grep -v '^$' train_full.tsv | head -n 20


In [ ]:
# from google.colab import runtime
# runtime.unassign()